In [ ]:
import os

# 1. Clone the repository manually
if not os.path.exists("VideoLLaMA2"):
    print("⬇️ Cloning the repository...")
    !git clone https://github.com/DAMO-NLP-SG/VideoLLaMA2.git
else:
    print("✅ Repository already exists.")

# 2. Enter folder and switch branch
%cd VideoLLaMA2
print("🔀 Switching to Audio-Visual branch...")
!git checkout audio_visual

# 3. THE FIX: Aggressively remove strict versions that break Colab
print("🛠️ Patching configuration files...")

# Fix 1: Unpin Numpy (The cause of your crash)
!sed -i 's/numpy==1.24.4/numpy/g' pyproject.toml
!sed -i 's/numpy==1.24.4/numpy/g' requirements.txt

# Fix 2: Remove Flash Attention (Incompatible with T4 GPU)
!sed -i '/flash-attn/d' pyproject.toml
!sed -i '/flash-attn/d' requirements.txt

# Fix 3: Unpin Torch (Prevents downgrading Colab's drivers)
!sed -i 's/torch==2.2.0/torch/g' requirements.txt
!sed -i 's/torchvision==0.17.0/torchvision/g' requirements.txt

# 4. Install dependencies manually first
print("📦 Installing dependencies (This may take 2 mins)...")
!pip install -r requirements.txt --quiet
!pip install ffmpeg-python bitsandbytes accelerate deepspeed --quiet

# 5. Install the package ignoring build isolation
print("🚀 Installing VideoLLaMA2...")
%pip install -e . --no-build-isolation

# 6. Verify and Go Back
%cd ..
import torch
try:
    from videollama2 import model_init, mm_infer
    print("\n✅ SUCCESS: Setup Complete! You can run Cell 2 now.")
except ImportError:
    print("\n❌ STILL FAILING: Try 'Runtime > Restart Session' and run this cell again.")

✅ Repository already exists.
/content/VideoLLaMA2
🔀 Switching to Audio-Visual branch...
M	pyproject.toml
M	requirements.txt
Already on 'audio_visual'
Your branch is up to date with 'origin/audio_visual'.
🛠️ Patching configuration files...
📦 Installing dependencies (This may take 2 mins)...
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.7/132.7 kB 5.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.3/7.3 MB 86.2 MB/s eta 0:00:00:00:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 21.4 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 728.5/728.5 MB 2.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 MB 6.8 MB/s eta 0:00:0000:0100:01
     ━━━━

In [ ]:
# Load the specific Audio-Visual model
# We use the 7B version because it fits on the free Colab T4 GPU
model_path = "DAMO-NLP-SG/VideoLLaMA2.1-7B-AV"

print("⏳ Downloading & Loading Model... (This takes a few minutes)")
disable_torch_init()

# The library automatically handles 8-bit loading to save RAM
model, processor, tokenizer = model_init(model_path)

print("✅ Model Loaded & Ready!")